In [ ]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import os
import io, requests
import joblib

# data_path = 'C:/Users/demo/Desktop/Lattitude/datas/'
data_path = 'datas'
os.makedirs(data_path, exist_ok=True)


# Types de voitures

In [ ]:
file_name = 'voitures-par-commune-par-energie'
ext = '.geojson'
url = 'http://opendata.agenceore.fr/explore/dataset/voitures-par-commune-par-energie/download?format=geojson&timezone=Europe/Berlin&use_labels_for_header=false'

try: 
    cars = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    # from url
    s = requests.get(url).content
    cars = gpd.read_file(io.StringIO(s.decode('utf-8')))

    # from local file
    # cars = gpd.read_file(os.path.join(data_path,file_name + ext))
   
    # avoiding multi dowloads for GES emmision lowering
    cars.to_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
cars.info()

In [ ]:
cars.isna().any()

In [ ]:
# cars.libepci # manque des valeurs

In [ ]:
cars.libgeo # complet mais a croiser avec les noms de communes

In [ ]:
cars.codgeo   ## LE BON CLIENT

In [ ]:
cars.drop(columns='geometry', inplace=True)

# Codes communes insee avec 


Juste pour poser un marqueur sur la commune

A remplacer avec decoupage administratif 

In [ ]:
file_name2 = 'laposte_hexasmal'
ext = '.geojson'
url ='https://www.data.gouv.fr/fr/datasets/r/7d6df627-a3e9-41cb-8914-573d46350e17'

try: 
    poste = gpd.read_feather(os.path.join(data_path,file_name2 + '.feather'))
except:
    poste = gpd.read_file(url)

    # s = requests.get(url).content
    # poste = gpd.read_file(io.StringIO(s.decode('utf-8')))

    # poste = gpd.read_file(os.path.join(data_path,file_name2 + ext))
    
    poste.to_feather(os.path.join(data_path,file_name2 + '.feather'))


In [ ]:
poste.info()

In [ ]:
poste.isna().any()

In [ ]:
poste.nom_de_la_commune.value_counts()

In [ ]:
cars.loc[0,:]

In [ ]:
poste.query('nom_de_la_commune == "INCHY"')

# Jointure des tables

In [ ]:
poste.rename(columns={'code_commune_insee': 'codgeo'}, inplace=True)

In [ ]:
# jointure des deux suivant le code commune insee
cars = cars.join(poste.set_index('codgeo')[['geometry']], on='codgeo')

In [ ]:
cars.info()

In [ ]:
cars.query("geometry == None")

In [ ]:
poste.query("codgeo == '09255'")

 Il y a un problème sur 1735 données de cars dont la geomtrie est None

 Pour l'instant drop

In [ ]:
cars = cars.query("geometry != None")
cars.head()

# Agrégations

In [ ]:
car_agg = cars.groupby('codgeo').agg(
                    {'nb_vp': ['min','max'],
                    'nb_vp_rechargeables_gaz': ['min','max'],
                    'nb_vp_rechargeables_el': ['min','max'],
                    'geometry': ['first']
                    }
                    ).reset_index()
car_agg.columns = car_agg.columns.to_flat_index()
car_agg.columns = [ x + '_' + y if x !='geometry' else x for x,y in car_agg.columns]
# car_agg.columns = ['min_nb_vp','max_nb_vp','min_nb_vp_rechargeables_gaz','max_nb_vp_rechargeables_gaz','min_nb_vp_rechargeables_el','max_nb_vp_rechargeables_el']
car_agg

# Markers

In [ ]:
from branca.colormap import linear, LinearColormap

# Define the color map
colors = ['#00ff00', '#ffff00', '#ff0000']
cmap_vp = LinearColormap(colors=colors, vmin=car_agg['nb_vp_max'].min(), vmax=car_agg['nb_vp_max'].max())
display(cmap_vp)

In [ ]:
nb_vp = MarkerCluster(name='Nombre de véhicules')
nb_el = MarkerCluster(name='Nombre de véhicules électriques')

for idx, row in car_agg.iterrows():
    # Clusters for vp_max circles
    nb_vp.add_child( folium.CircleMarker(
                            location = [row.geometry.y, row.geometry.x],
                            radius = 50,
                            popup = f'<b>véhicules: </b> {row["nb_vp_max"]}',
                            color = cmap_vp(row['nb_vp_max']),
                            fill = True,
                            fill_color = cmap_vp(row['nb_vp_max']),
                            fillOpacity = .1,
                      )
                    )    
    nb_el.add_child( folium.Marker(
                            location = [row.geometry.y, row.geometry.x],
                            popup = f'<b>électriques: </b> {row["nb_vp_rechargeables_el_max"]}',
                            icon = folium.Icon(icon="info-sign", color='blue'),
                      )
                    )


In [ ]:
marker_path = 'datas/markers'
os.makedirs(marker_path, exist_ok=True)

joblib.dump(nb_vp, os.path.join(marker_path,'nombre_vehicules.mrkr'))

joblib.dump(nb_el, os.path.join(marker_path,'nombre_vehicules_electriques.mrkr'))


# Carte

In [ ]:
map = folium.Map(location= [48.8, 2.3],
                    width='100%', height='100%', left='00%', top='0%', position='relative', 
                    tiles= 'OpenStreetMap',     #OpenStreetMap', Stamen
                    attr=None,    #  None   Terrain, Toner, and Watercolor
                    min_zoom=0, max_zoom=18, zoom_start=10, 
                    min_lat=- 90, max_lat=90, min_lon=- 180, max_lon=180, max_bounds=False, 
                    crs='EPSG3857', control_scale=False, prefer_canvas=False, 
                    no_touch=False, disable_3d=False, png_enabled=False, zoom_control=True)

nb_vp.add_to(map)
nb_el.add_to(map)
# trafic.add_to(all_in)   
           
folium.LayerControl().add_to(map)

map

In [ ]:
maps_path = 'datas/maps'
os.makedirs(maps_path, exist_ok=True)


map.save(os.path.join(maps_path,'carte_vehicules.html'))